<a href="https://colab.research.google.com/github/skyjadel/Contrail-Detection/blob/main/Titanic_Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/train.csv')

In [2]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


先移除 Name，Ticket，Cabin<br>


In [3]:
df = df[list(set(df.columns).difference(['Ticket','Name','Cabin']))]
df

,Parch,Embarked,Fare,Sex,Survived,PassengerId,Pclass,SibSp,Age
0,0,S,7.2500,male,0,1,3,1,22.0
1,0,C,71.2833,female,1,2,1,1,38.0
2,0,S,7.9250,female,1,3,3,0,26.0
3,0,S,53.1000,female,1,4,1,1,35.0
4,0,S,8.0500,male,0,5,3,0,35.0
...,...,...,...,...,...,...,...,...,...
886,0,S,13.0000,male,0,887,2,0,27.0
887,0,S,30.0000,female,1,888,1,0,19.0
888,2,S,23.4500,female,0,889,3,1,NaN
889,0,C,30.0000,male,1,890,1,0,26.0


# 資料前處理

## 區分 Training Set 與 Testing Set

In [4]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=109)
print(f'{len(train_df)} rows in training set.')
print(f'{len(test_df)} rows in testing set.')

712 rows in training set.
179 rows in testing set.


## 補缺失值

從上次的EDA我們知道，有缺失值的欄位有 Age 和 Embarked，Age將補上training set 的平均值，Embarked將補上trainin set 的最多分類。


---

20240510 12:40 <br>
因為發現不同艙等的年齡分布有差別，而三等艙的旅客年齡缺失的機率又比較大，所以改填各艙等的平均年齡。

In [5]:
# 找出哪個港口上船的人最多
grouped = train_df.groupby('Embarked')
harbor_counts = {g[0]:len(g[1]) for g in grouped}

max_count = 0
for k, v in harbor_counts.items():
  if v > max_count:
    max_count = v
    major_embarked_harbor = k

In [6]:
# Training Set 的年齡平均值
#mean_age = train_df['Age'].mean()
mean_age = {i: train_df[train_df['Pclass']==i]['Age'].mean() for i in range(1,4)}

In [7]:
def fill_nan(df, age_value=mean_age, embarked_value=major_embarked_harbor):
  df = df.fillna({'Embarked':embarked_value})


  if type(age_value) == dict:
    age_is_na = df['Age'].isna()
    for i in df.index:
      if age_is_na[i]:
        df.loc[i, 'Age'] = mean_age[df.loc[i, 'Pclass']]
  else:
    df = df.fillna({'Age':age_value})

  return df

## Labeling

Sex 欄位改稱 Sex_female 欄位 <br>
Embarked 使用onehot

In [8]:
def labeling(df):
  df = pd.concat([df, pd.get_dummies(df[['Sex', 'Embarked']])], axis=1)
  df.drop(['Sex', 'Embarked', 'Sex_male'], axis=1, inplace=True)
  return df

## 添加特徵

前面發現單人登船者生還率較低，因此嘗試加入是否為單人特徵
(測試結果發現沒有比較好，停用)

In [9]:
def single(df):
  df['Single'] = 0
  for i in df.index:
    if sum(df.loc[i][['SibSp', 'Parch']])==0:
      df.loc[i, 'Single'] = 1
  return df

前面發現15歲以下生還率較高，因此嘗試加入是否為15歲以下特徵

In [10]:
def young(df):
  df['Young'] = 0
  for i in df.index:
    if df.loc[i]['Age']<=15:
      df.loc[i, 'Young'] = 1
  return df

## 前處理函式

In [11]:
def preprocess(df):
  df = labeling(fill_nan(df))
  #df = single(df)
  df = young(df)
  return df

In [12]:
train_df = preprocess(train_df)
test_df = preprocess(test_df)

In [13]:
train_df

,Parch,Fare,Survived,PassengerId,Pclass,SibSp,Age,Sex_female,Embarked_C,Embarked_Q,Embarked_S,Young
70,0,10.5000,0,71,2,0,32.000000,False,False,False,True,0
183,1,39.0000,1,184,2,2,1.000000,False,False,False,True,1
599,0,56.9292,1,600,1,1,49.000000,False,True,False,False,0
815,0,0.0000,0,816,1,0,38.637973,False,False,False,True,0
674,0,0.0000,0,675,2,0,29.525926,False,False,False,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...
399,0,12.6500,1,400,2,0,28.000000,True,False,False,True,0
141,0,7.7500,1,142,3,0,22.000000,True,False,False,True,0
757,0,11.5000,0,758,2,0,18.000000,False,False,False,True,0
245,0,90.0000,0,246,1,2,44.000000,False,False,True,False,0


# 訓練

In [14]:
columns_X = list(set(train_df.columns) - {'Survived', 'PassengerId'})
columns_y = ['Survived']

train_X = train_df[columns_X]
train_y = train_df[columns_y]
test_X = test_df[columns_X]
test_y = test_df[columns_y]

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)
print(f'平均準確率{np.mean(scores):.4f}')

[0.74125874 0.84615385 0.84507042 0.72535211 0.87323944]
平均準確率0.8062


In [16]:
H = log.fit(np.array(train_X), np.array(train_y).reshape(-1))

In [17]:
pred_y = log.predict(np.array(train_X))
true_y = yt = np.array(train_y).reshape(-1)
acc_samples = sum(true_y * pred_y) + sum((1-true_y)*(1-pred_y))
print(f'Accuracy for training set is {acc_samples/len(train_df):.4f} ({acc_samples}/{len(train_df)})')

Accuracy for training set is 0.8118 (578/712)


In [18]:
pred_y = log.predict(np.array(test_X))
true_y = yt = np.array(test_y).reshape(-1)
acc_samples = sum(true_y * pred_y) + sum((1-true_y)*(1-pred_y))
print(f'Accuracy for testing set is {acc_samples/len(test_df):.4f} ({acc_samples}/{len(test_df)})')

Accuracy for testing set is 0.8156 (146/179)
